# Retweets Networks

In this notebook, we create a daily retweet network. First, we transform our dataset into a Source-Target DataFrame. Afterward, we construct a `graph-tool` object.

We have 45,330,718 Tweets from 37330 from which 30,918,011 are ReTweets

In [26]:
import pandas as pd
import numpy as np
import pickle
import os
from tqdm import tqdm
import scipy.sparse as sp
from scipy.sparse import lil_matrix
from scipy.sparse import csr_matrix
from scipy.sparse import find
import scipy.sparse
import graph_tool.all as gt
import glob

In [38]:
save_path = "../../../Data/3_Day_Graphs"

## Retweet network with a 3-day rolling window

In [28]:
tweets = pd.read_pickle('../../../Data/Tweets_DataFrames/tweets_lite.gzip', compression='gzip')
print('Shape:', tweets.shape)
tweets.head()

Shape: (45330718, 4)


,Author ID,Date,Reference Type,Referenced Tweet Author ID
0,9.180596e+17,2021/06/29 23:37:59,retweeted,1.419439e+08
1,9.180596e+17,2021/06/29 23:37:16,retweeted,8.628063e+17
2,9.180596e+17,2021/06/29 23:31:36,retweeted,1.402301e+18
3,9.180596e+17,2021/06/29 23:30:41,retweeted,3.824198e+08
4,9.180596e+17,2021/06/29 23:29:39,retweeted,1.483430e+07


In [34]:
retweets = tweets[tweets['Reference Type'] == 'retweeted']
retweets = retweets.drop(columns=['Reference Type'])
retweets["Date"] = pd.to_datetime(retweets["Date"]).dt.date
print(f'Tenemos :{retweets.shape[0]} Retweets')
retweets.head()

Tenemos :30918011 Retweets


,Author ID,Date,Referenced Tweet Author ID
0,9.180596e+17,2021-06-29,1.419439e+08
1,9.180596e+17,2021-06-29,8.628063e+17
2,9.180596e+17,2021-06-29,1.402301e+18
3,9.180596e+17,2021-06-29,3.824198e+08
4,9.180596e+17,2021-06-29,1.483430e+07


In [35]:
# List of Twitter users
users = np.unique(retweets[['Author ID']].values)
users = [ int(x) for x in users ]

# Dates of the Paro Nacional
v1_start = '2021-04-28 00:00:00'
v1_end = '2021-06-27 00:00:00'
date_start = pd.date_range(start = v1_start, end = v1_end, freq = 'D')

v2_start = '2021-04-30 23:59:59'
v2_end = '2021-06-29 23:59:59'
date_end = pd.date_range(start = v2_start, end = v2_end, freq = 'D')

user_indices = {user: idx for idx, user in enumerate(users)}
datestr = list(date_end.strftime("%d-%m-%Y"))

In [36]:
# Import political labelling dictionary
user_to_party_paro = pd.read_pickle("../../../Data/Pickle/user_to_party_paro.pkl")

ids_faltantes1 = set(retweets["Author ID"]) - set(user_to_party_paro.keys())
ids_faltantes2 = set(retweets["Referenced Tweet Author ID"]) - set(user_to_party_paro.keys())
ids_faltantes = np.concatenate((list(ids_faltantes1), list(ids_faltantes2)))
print(f"Faltan clasificar {len(np.unique(ids_faltantes))} usuarios")

# Los dejamos como inclasificados 
for usuario in ids_faltantes:
    user_to_party_paro[usuario] = np.nan

retweets.head()

Faltan clasificar 587246 usuarios


,Author ID,Date,Referenced Tweet Author ID
0,9.180596e+17,2021-06-29,1.419439e+08
1,9.180596e+17,2021-06-29,8.628063e+17
2,9.180596e+17,2021-06-29,1.402301e+18
3,9.180596e+17,2021-06-29,3.824198e+08
4,9.180596e+17,2021-06-29,1.483430e+07


In [42]:
for d_s, d_e in tqdm(zip(date_start, date_end)):
    # Select the retweets from the desired date
    temp = retweets[(retweets['Date'] >= d_s.date()) & (retweets['Date'] < d_e.date())]

    # Get total of Tweets during that period
    rts_per_user = temp.groupby('Author ID').size().reset_index(name = 'total')

    # Join
    temp = temp.merge(rts_per_user, how = 'right', on='Author ID')
    temp = temp.groupby(["Author ID", "Referenced Tweet Author ID"]).size().reset_index(name = "w")
    temp.columns = ["source", "target", "w"]

    # Assign political affiliation
    temp["source_political_afilliation"] = temp["source"].apply(lambda x: user_to_party_paro[x])
    temp["target_political_afilliation"] = temp["target"].apply(lambda x: user_to_party_paro[x])
    
    # Save results as csv
    temp.to_csv(os.path.join(save_path, "Source-Target", str(d_e.date()) + ".csv"), index = False, sep = ";")

61it [05:34,  5.49s/it]


In [47]:
nodes_no_receipt = set(retweets["Author ID"]) -  set(retweets["Referenced Tweet Author ID"])
nodes_no_send = set(retweets["Referenced Tweet Author ID"]) - set(retweets["Author ID"])
self_loops = set(retweets[retweets['Author ID'] == retweets['Referenced Tweet Author ID']])

print(f'Nodos que Retweeteados pero que no Retweetean: {len(nodes_no_receipt)}')
print(f'Nodos que Retweetean a alguien pero no Son Retweeteados: {len(nodes_no_send)}')
print(f'Nodos que tienen un self loop: {len(self_loops)}')

nodes_id = list(set(retweets["Author ID"]).union(set(retweets["Referenced Tweet Author ID"])))
print(f'Total de nodos en la muestra: {len(nodes_id)}')

Nodos que Retweeteados pero que no Retweetean: 6658
Nodos que Retweetean a alguien pero no Son Retweeteados: 587712
Nodos que tienen un self loop: 3
Total de nodos en la muestra: 624358


In [51]:
nodes_political_affilitation = []

for i in tqdm(nodes_id):
    # Assign Political Affiliation
    pa = user_to_party_paro.get(i, np.nan) # using .get() for safe dictionary lookup
    nodes_political_affilitation.append(pa)

num_nodes = len(nodes_id)
nodes_idx = [i for i in range(num_nodes)]

# Create dictionary
nodes_dict = {
    'id': nodes_id,
    'idx': nodes_idx,
    'pa': nodes_political_affilitation
}

# Save as pickle
with open("../../../Data/3_Day_Graphs/nodes_dict.pkl", "wb") as f:
    pickle.dump(nodes_dict, f)

100%|██████████| 624358/624358 [00:00<00:00, 1333630.36it/s]


In [178]:
#SANDBOX
# Sorce-target ex
data = {
    'source': [1,3,4,5],
    'source_label': ['par' if i%2 == 0  else 'impar' for i in [1,3,4,5]],
    'target': [9,8,7,6],
    'target_label': ['par' if i%2 == 0  else 'impar' for i in [9,8,7,6]],
    'weight': [1,2,3,4],
}

Df = pd.DataFrame(data)
s_t = Df[['source', 'target']]
s_t = [ tuple(row) for row in s_t.itertuples(index=False)]

g = gt.Graph(directed = True)
g.add_edge_list(s_t)

modulo_2 = g.new_edge_property('double')
for i, edge in enumerate(g.get_edges()):
    print(f'Property of {edge} is {Df.weight[i]}. Type of Edge: {type(edge)}')
    modulo_2[edge] = Df.weight[i]

print(modulo_2)
g.edge_properties['modulo_2'] = modulo_2
g

Property of [1 9] is 1. Type of Edge: <class 'numpy.ndarray'>
Property of [3 8] is 2. Type of Edge: <class 'numpy.ndarray'>
Property of [4 7] is 3. Type of Edge: <class 'numpy.ndarray'>
Property of [5 6] is 4. Type of Edge: <class 'numpy.ndarray'>
<EdgePropertyMap object with value type 'double', for Graph 0x7f285a6f8b90, at 0x7f2a96756890>


<Graph object, directed, with 10 vertices and 4 edges, 1 internal edge property, at 0x7f285a6f8b90>

In [65]:
len(set([1,3,4,5,6,3,6,7,8]).union(set([9,8,7,6,5,4,3,2,1])))

9

In [174]:
import graph_tool.all as gt

# Create a graph
g = gt.Graph(directed=False)
v1 = g.add_vertex()
v2 = g.add_vertex()
v3 = g.add_vertex()
v4 = g.add_vertex()

# Add edges
e1 = g.add_edge(v1, v2)
e2 = g.add_edge(v2, v3)
e3 = g.add_edge(v3, v4)

# Create an edge property map for weights
edge_weight_map = g.new_edge_property("double")

# Create a Vertex property map for labels
vertex_label_map = g.new_vertex_property('string')

# Assign weights to the edges using a for loop
edge_list = g.get_edges()
weights = [5.0, 3.2, 7.1]  # Example weights
for i, e in enumerate(edge_list):
    print(f'Property of {e} is {weights[i]}. Type of Edge: {type(e)}')
    edge_weight_map[e] = weights[i]

# Assign Labels to vertices using loop
vertex_list = g.get_vertices()
labels = ['par', 'impar', 'par', 'impar']
for i,v in enumerate(vertex_list):
    print(i,v)
    print(f'Property of {v} is {labels[i]}. Type of Edge: {type(v)}')
    vertex_label_map[v] = labels[i]

# Access and print the weights for each edge
for v in g.vertices():
    print(f"Label of Vertex {v}: {vertex_label_map[v]}")

g

Property of [0 1] is 5.0. Type of Edge: <class 'numpy.ndarray'>
Property of [1 2] is 3.2. Type of Edge: <class 'numpy.ndarray'>
Property of [2 3] is 7.1. Type of Edge: <class 'numpy.ndarray'>
0 0
Property of 0 is par. Type of Edge: <class 'numpy.int64'>
1 1
Property of 1 is impar. Type of Edge: <class 'numpy.int64'>
2 2
Property of 2 is par. Type of Edge: <class 'numpy.int64'>
3 3
Property of 3 is impar. Type of Edge: <class 'numpy.int64'>
Label of Vertex 0: par
Label of Vertex 1: impar
Label of Vertex 2: par
Label of Vertex 3: impar


<Graph object, undirected, with 4 vertices and 3 edges, at 0x7f285a6b1a50>

In [ ]:
# Create a dictionary for O(1) lookup time of indices
id_to_idx = {node_id: idx for idx, node_id in enumerate(nodes_dict['id'])}

for d_s, d_e in tqdm(zip(date_start, date_end)):
    # Filter retweets first
    filtered_retweets = retweets[(retweets['Date'] >= d_s.date()) & (retweets['Date'] < d_e.date())]
    
    # Create weights
    test = filtered_retweets.groupby(['Author ID', 'Referenced Tweet Author ID']).size().reset_index(name="w")
    
    # Use dictionary lookups for index mappings
    test['idx_s'] = [id_to_idx.get(x, np.nan) for x in test['Author ID']]
    test['idx_t'] = [id_to_idx.get(x, np.nan) for x in test['Referenced Tweet Author ID']]
    
    # The edge list should be based on the idx of the nodes and include weights
    edges_list_with_weights = list(zip(test['idx_s'].dropna().astype(int), test['idx_t'].dropna().astype(int), test['w']))
    
    # Create graph
    g = gt.Graph(directed=True)
    g.add_vertex(num_nodes)
    
    # Node attributes
    node_id = g.new_vertex_property("string", vals=nodes_dict['id'])
    node_label = g.new_vertex_property("string", vals=nodes_dict['label'])
    node_affiliation = g.new_vertex_property("string", vals=nodes_dict['pa'])
    
    # Assign attributes to graph
    g.vertex_properties["ID"] = node_id
    g.vertex_properties["Label"] = node_label
    g.vertex_properties["Political Affiliation"] = node_affiliation
    
    # Edge weight attribute
    edge_weights = g.new_edge_property("int")
    
    # Add edges and assign weights
    for source, target, weight in edges_list_with_weights:
        e = g.add_edge(source, target)
        edge_weights[e] = weight

    g.edge_properties["weight"] = edge_weights
    
    g.save(os.path.join(save_path, str(d.date()) + ".graphml"))

## Daily retweet network (with no rolling window)

In [3]:
tweets_paro = glob.glob('/mnt/disk2/Data/Tweets_DataFrames/tweets_paro_*')
tweets_paro

['/mnt/disk2/Data/Tweets_DataFrames/tweets_paro_4.gzip',
 '/mnt/disk2/Data/Tweets_DataFrames/tweets_paro_5.gzip',
 '/mnt/disk2/Data/Tweets_DataFrames/tweets_paro_3.gzip',
 '/mnt/disk2/Data/Tweets_DataFrames/tweets_paro_2.gzip',
 '/mnt/disk2/Data/Tweets_DataFrames/tweets_paro_1.gzip']

In [4]:
# Import the tweets from Paro but select only the retweets
retweets = pd.DataFrame()
for file in tqdm(tweets_paro):
    tweets = pd.read_pickle(file, compression = "gzip")

    # Select only retweets
    rts = tweets.loc[tweets["Reference Type"] == "retweeted",:].reset_index(drop = True)
    rts = rts.drop(columns = 'Reference Type')
    retweets = pd.concat([retweets, rts], axis = 0)
retweets = retweets.reset_index(drop = True)
del rts, tweets 
print('Shape:', retweets.shape)

100%|██████████| 5/5 [03:44<00:00, 44.84s/it]


Shape: (30918011, 24)


In [5]:
retweets.head()

,ID,Permalink,Author ID,Author Name,Author Location,Author Description,Author Followers,Author Following,Author Tweets,Author Profile Image,...,Retweets,Favorites,Quotes,is Retweet?,Reply To User Name,Mentions,Referenced Tweet,Referenced Tweet Author ID,Media URLs,Media Keys
0,1.409944e+18,/wilman_86/status/1409944432011628548,325932532.0,wilman_86,Colombia,Gregario del día a día. El mas combativo en lo...,609.0,1139.0,65944.0,https://pbs.twimg.com/profile_images/160707892...,...,131.0,0.0,0.0,True,NaN,udsnoexisten,1.409943e+18,2.195296e+07,NaN,NaN
1,1.409726e+18,/wilman_86/status/1409725681546760200,325932532.0,wilman_86,Colombia,Gregario del día a día. El mas combativo en lo...,609.0,1139.0,65944.0,https://pbs.twimg.com/profile_images/160707892...,...,1.0,0.0,0.0,True,NaN,charlasamenas63,1.409726e+18,9.273397e+17,NaN,NaN
2,1.409498e+18,/wilman_86/status/1409498069096353800,325932532.0,wilman_86,Colombia,Gregario del día a día. El mas combativo en lo...,609.0,1139.0,65944.0,https://pbs.twimg.com/profile_images/160707892...,...,1.0,0.0,0.0,True,NaN,elyrxc,1.409352e+18,1.402825e+18,https://pbs.twimg.com/ext_tw_video_thumb/14093...,7_1409351854983725057
3,1.409266e+18,/wilman_86/status/1409265731489964032,325932532.0,wilman_86,Colombia,Gregario del día a día. El mas combativo en lo...,609.0,1139.0,65944.0,https://pbs.twimg.com/profile_images/160707892...,...,84.0,0.0,0.0,True,NaN,Lidio_Dominante,1.409202e+18,1.391738e+18,NaN,NaN
4,1.408904e+18,/wilman_86/status/1408904423372378112,325932532.0,wilman_86,Colombia,Gregario del día a día. El mas combativo en lo...,609.0,1139.0,65944.0,https://pbs.twimg.com/profile_images/160707892...,...,17.0,0.0,0.0,True,NaN,Lidio_Dominante,1.408853e+18,1.391738e+18,NaN,NaN


In [7]:
# Fix data types
retweets["ID"] = retweets["ID"].astype(int)
retweets["Author ID"] = retweets["Author ID"].astype(int)
retweets['Referenced Tweet Author ID'] = retweets['Referenced Tweet Author ID'].astype(int)
retweets['Referenced Tweet'] = retweets['Referenced Tweet'].astype(int)

# Remove time from retweets date
retweets["Date"] = pd.to_datetime(retweets["Date"]).dt.date

In [8]:
# Import political labelling dictionary
user_to_party_paro = pd.read_pickle("/mnt/disk2/Data/Pickle/user_to_party_paro.pkl")

ids_faltantes1 = set(retweets["Author ID"]) - set(user_to_party_paro.keys())
ids_faltantes2 = set(retweets["Referenced Tweet Author ID"]) - set(user_to_party_paro.keys())
ids_faltantes = np.concatenate((list(ids_faltantes1), list(ids_faltantes2)))
print(f"Faltan clasificar {len(np.unique(ids_faltantes))} usuarios")

# Los dejamos como inclasificados 
for usuario in ids_faltantes:
    user_to_party_paro[usuario] = np.nan

Faltan clasificar 587246 usuarios


### Create Source-Target DataFrame

In [9]:
save_path = "../../Data/Daily_graphs"

In [17]:
fecha_min = retweets["Date"].min()
fecha_max = retweets["Date"].max()
print(f"We have retweets from {fecha_min.strftime('%Y-%m-%d')} to {fecha_max.strftime('%Y-%m-%d')}")
daily_grid = pd.date_range(start = fecha_min, end = fecha_max, freq = 'D')

We have retweets from 2021-04-28 to 2021-06-29


In [11]:
users_information = pd.read_pickle("../../../Data/Tweets_DataFrames/users_information.gzip", compression = "gzip")

In [12]:
users_information

Author Location  \
Author ID    Author Name                                
0.000000e+00 0               True, False, True, False   
1.000000e+00 0                                  False   
2.000000e+00 0                                  False   
3.000000e+00 0                            False, True   
             1                                  False   
...                                               ...   
1.389722e+18 Neoplasticista                  Colombia   
1.389737e+18 JC13177979        Bogotá, D.C., Colombia   
1.389741e+18 JhonatanVRojo         Medellín, Colombia   
1.389769e+18 VaneLen18                       Colombia   
1.389784e+18 kars0518                            COL.   

                                                            Author Description  \
Author ID    Author Name                                                         
0.000000e+00 0                      nan, equidad_mujer, mariumega, PattyRosi24   
1.000000e+00 0                                              nan, equidad_mujer   
2.000000e+00 0               nan, estoacaquees, thearchipielago, equidad_mu...   
3.000000e+00 0                           MinTransporteCo, nan, Supertransporte   
             1                                                             nan   
...                                                                        ...   
1.389722e+18 Neoplasticista                      Arquitecto. Contra Corriente.   
1.389737e+18 JC13177979      Aunque nadie ha podido regresar y hacer un nue...   
1.389741e+18 JhonatanVRojo                 El mundo es más que blanco & negro.   
1.389769e+18 VaneLen18                                                     nan   
1.389784e+18 kars0518                                                      nan   

                             Author Followers  Author Following  \
Author ID    Author Name                                          
0.000000e+00 0                            NaN      1.384355e+18   
1.000000e+00 0                            NaN      1.398276e+18   
2.000000e+00 0                            NaN      1.396578e+18   
3.000000e+00 0                            NaN      1.396701e+18   
             1                            NaN               NaN   
...                                       ...               ...   
1.389722e+18 Neoplasticista              91.0      4.980000e+02   
1.389737e+18 JC13177979                  94.0      1.780000e+02   
1.389741e+18 JhonatanVRojo              103.0      4.270000e+02   
1.389769e+18 VaneLen18                    8.0      9.300000e+01   
1.389784e+18 kars0518                    53.0      8.700000e+01   

                             Author Tweets  \
Author ID    Author Name                     
0.000000e+00 0                         NaN   
1.000000e+00 0                         NaN   
2.000000e+00 0                         NaN   
3.000000e+00 0                         NaN   
             1                         NaN   
...                                    ...   
1.389722e+18 Neoplasticista         3534.0   
1.389737e+18 JC13177979             7083.0   
1.389741e+18 JhonatanVRojo          1257.0   
1.389769e+18 VaneLen18              1179.0   
1.389784e+18 kars0518                768.0   

                                                               Author Verified  
Author ID    Author Name                                                        
0.000000e+00 0               nan, https://pbs.twimg.com/media/E2C38-kXEAIkE...  
1.000000e+00 0               nan, https://pbs.twimg.com/media/E19UAtWXsAYjz...  
2.000000e+00 0               https://pbs.twimg.com/ext_tw_video_thumb/14011...  
3.000000e+00 0               https://pbs.twimg.com/media/E0ysfTuWUAUo8by.jp...  
             1                 https://pbs.twimg.com/media/E1dlhtaXEAYTwol.jpg  
...                                                                        ...  
1.389722e+18 Neoplasticista                                              False  
1.389737e+18 JC13177979      

In [30]:
# Label Referenced Tweet Author
retweets = users_information.reset_index()[["Author ID", "Author Name"]] \
    .rename(columns = {"Author ID": "Referenced Tweet Author ID", 
                       "Author Name": "Referenced Tweet Author Name"}) \
                       .merge(retweets, how = "right", on = "Referenced Tweet Author ID")

In [31]:
retweets.head()

,Referenced Tweet Author ID,Referenced Tweet Author Name,ID,Permalink,Author ID,Author Name,Author Location,Author Description,Author Followers,Author Following,...,Replies,Retweets,Favorites,Quotes,is Retweet?,Reply To User Name,Mentions,Referenced Tweet,Media URLs,Media Keys
0,2.195296e+07,NaN,1409944432011628544,/wilman_86/status/1409944432011628548,325932532,wilman_86,Colombia,Gregario del día a día. El mas combativo en lo...,609.0,1139.0,...,0.0,131.0,0.0,0.0,True,NaN,udsnoexisten,1409942622140088320,NaN,NaN
1,9.273397e+17,charlasamenas63,1409725681546760192,/wilman_86/status/1409725681546760200,325932532,wilman_86,Colombia,Gregario del día a día. El mas combativo en lo...,609.0,1139.0,...,0.0,1.0,0.0,0.0,True,NaN,charlasamenas63,1409725505893457920,NaN,NaN
2,1.402825e+18,NaN,1409498069096353792,/wilman_86/status/1409498069096353800,325932532,wilman_86,Colombia,Gregario del día a día. El mas combativo en lo...,609.0,1139.0,...,0.0,1.0,0.0,0.0,True,NaN,elyrxc,1409351876420771840,https://pbs.twimg.com/ext_tw_video_thumb/14093...,7_1409351854983725057
3,1.391738e+18,NaN,1409265731489964032,/wilman_86/status/1409265731489964032,325932532,wilman_86,Colombia,Gregario del día a día. El mas combativo en lo...,609.0,1139.0,...,0.0,84.0,0.0,0.0,True,NaN,Lidio_Dominante,1409202187637510144,NaN,NaN
4,1.391738e+18,NaN,1408904423372378112,/wilman_86/status/1408904423372378112,325932532,wilman_86,Colombia,Gregario del día a día. El mas combativo en lo...,609.0,1139.0,...,0.0,17.0,0.0,0.0,True,NaN,Lidio_Dominante,1408852871446253568,NaN,NaN


In [33]:
# Users without name
retweets.iloc[:, 0:2].drop_duplicates().isna().sum()

Referenced Tweet Author ID           0
Referenced Tweet Author Name    587246
dtype: int64

In [46]:
for d in tqdm(daily_grid):
    # Select the retweets from the desired date
    temp = retweets[retweets['Date'] == d.date()]
    temp = temp.groupby(["Author ID", "Author Name", "Date", "Referenced Tweet Author ID", "Referenced Tweet Author Name"]).size().reset_index(name = "w")
    temp.columns = ["source", "source_label", "date", "target", "target_label", "w"]
    # Assign political affiliation
    temp["source_political_afilliation"] = temp["source"].apply(lambda x: user_to_party_paro[x])
    temp["target_political_afilliation"] = temp["target"].apply(lambda x: user_to_party_paro[x])
    # Save results as csv
    temp.to_csv(os.path.join(save_path, "Source-Target", str(d.date()) + ".csv"), index = False, sep = ";")

100%|██████████| 63/63 [03:11<00:00,  3.04s/it]


### Create networks

In [54]:
# Node list
nodes_id = list(set(retweets["Author ID"]).union(set(retweets["Referenced Tweet Author ID"])))
len(nodes_id)

624358

In [55]:
users_information.shape

(37344, 6)

In [72]:
users_information = users_information.reset_index()

In [207]:
# Convert users_information DataFrame to a dictionary for fast lookup
users_info_dict = users_information.set_index('Author ID')['Author Name'].to_dict()

nodes_label = []
nodes_political_affilitation = []
for i in tqdm(nodes_id):
    # Assign label
    label = users_info_dict.get(i, np.nan) # using .get() for safe dictionary lookup
    nodes_label.append(label)

    # Assign Political Affiliation
    pa = user_to_party_paro.get(i, np.nan) # using .get() for safe dictionary lookup
    nodes_political_affilitation.append(pa)

num_nodes = len(nodes_id)
nodes_idx = [i for i in range(num_nodes)]

# Create dictionary
nodes_dict = {
    'id': nodes_id,
    'idx': nodes_idx,
    'label': nodes_label,
    'pa': nodes_political_affilitation
}

# Save as pickle
with open("../../../Data/Daily_graphs/nodes_dict.pkl", "wb") as f:
    pickle.dump(nodes_dict, f)

  0%|          | 0/624358 [00:00<?, ?it/s]

100%|██████████| 624358/624358 [00:01<00:00, 429591.22it/s]


In [215]:
# Create a dictionary for O(1) lookup time of indices
id_to_idx = {node_id: idx for idx, node_id in enumerate(nodes_dict['id'])}

for d in tqdm(daily_grid):
    # Filter retweets first
    filtered_retweets = retweets[retweets['Date'] == d.date()]
    
    # Create weights
    test = filtered_retweets.groupby(['Author ID', 'Referenced Tweet Author ID']).size().reset_index(name="w")
    
    # Use dictionary lookups for index mappings
    test['idx_s'] = [id_to_idx.get(x, np.nan) for x in test['Author ID']]
    test['idx_t'] = [id_to_idx.get(x, np.nan) for x in test['Referenced Tweet Author ID']]
    
    # The edge list should be based on the idx of the nodes and include weights
    edges_list_with_weights = list(zip(test['idx_s'].dropna().astype(int), test['idx_t'].dropna().astype(int), test['w']))
    
    # Create graph
    g = gt.Graph(directed=True)
    g.add_vertex(num_nodes)
    
    # Node attributes
    node_id = g.new_vertex_property("string", vals=nodes_dict['id'])
    node_label = g.new_vertex_property("string", vals=nodes_dict['label'])
    node_affiliation = g.new_vertex_property("string", vals=nodes_dict['pa'])
    
    # Assign attributes to graph
    g.vertex_properties["ID"] = node_id
    g.vertex_properties["Label"] = node_label
    g.vertex_properties["Political Affiliation"] = node_affiliation
    
    # Edge weight attribute
    edge_weights = g.new_edge_property("int")
    
    # Add edges and assign weights
    for source, target, weight in edges_list_with_weights:
        e = g.add_edge(source, target)
        edge_weights[e] = weight

    g.edge_properties["weight"] = edge_weights
    
    g.save(os.path.join(save_path, str(d.date()) + ".graphml"))

 33%|███▎      | 21/63 [19:38<36:55, 52.75s/it]

In [ ]:
print(g)
print(g.vp["ID"][624353])
print(g.vp["Label"][624353])
print(g.vp["Political Affiliation"][624353])

<Graph object, directed, with 624358 vertices and 460845 edges, 3 internal vertex properties, at 0x7f08b17f8150>
113246190
renlopezs
Retweets Centro
